<a href="https://colab.research.google.com/github/fcrmhmd/K-Nearest-Neighbors/blob/main/KNN_Keluhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT & LOAD LIB DATA

In [ ]:
!pip install Sastrawi
!pip install 'fsspec>=0.3.3'


     |████████████████████████████████| 209 kB 5.5 MB/s 
     |████████████████████████████████| 134 kB 5.2 MB/s 


In [ ]:
import pickle
import pandas as pd
import re
import string
import unicodedata
import nltk
from bs4 import BeautifulSoup

from emo_unicode import UNICODE_EMO, EMOTICONS
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords



nltk.download('punkt')
nltk.download('stopwords')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import time
import dask.dataframe as dd
from dask.multiprocessing import get

ModuleNotFoundError: ignored

# PREPROCESSING

In [ ]:
class TextPreprocessing:
    def __init__(self, text="test"):
        self.text = text

    def lowercase(self):
        """Convert to lowercase"""
        self.text = str(self.text).lower()
        self.text = self.text.strip()
        return self

    def strip_html(self):
        """Stopword removal"""
        soup = BeautifulSoup(self.text, "html.parser")
        self.text = soup.get_text()
        return self

    def remove_url(self):
        """Remove URL (http/https/www) or custom URL"""
        self.text = re.sub(r"https?://\S+|www\.\S+", "", self.text)
        return self

    def remove_email(self):
        """Remove email"""
        self.text = re.sub("\S*@\S*\s?", "", self.text)
        return self

    def remove_between_square_brackets(self):
        """Remove string beetwen square brackets []"""
        self.text = re.sub("\[[^]]*\]", "", self.text)
        return self

    def remove_numbers(self):
        """Remove numbers"""
        self.text = re.sub("[-+]?[0-9]+", "", self.text)
        return self

    def remove_emoji(self):
        """Remove emoji, e.g 😜😀 """
        emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            "]+",
            flags=re.UNICODE,
        )
        self.text = emoji_pattern.sub(r"", self.text)
        return self

    def remove_emoticon(self):
        """Remove emoticon, e.g :-)"""
        emoticon_pattern = re.compile(u"(" + u"|".join(k for k in EMOTICONS) + u")")
        self.text = emoticon_pattern.sub(r"", self.text)
        return self
    
    ##def convert_emoji(self):
    ##    """Convert emoji to word"""
    ##    for emoji in UNICODE_EMO:
    ##        self.text = self.text.replace(
    ##            emoji,
    ##            "_".join(UNICODE_EMO[emoji].replace(",", "").replace(":", "").split()),
    ##        )
    ##    return self

    ##def convert_emoticon(self):
    ##    """Convert emoticon to word"""
    ##    for emoticon in EMOTICONS:
    ##        self.text = re.sub(
    ##            u"(" + emoticon + ")",
    ##            "_".join(EMOTICONS[emoticon].replace(",", "").split()),
    ##            self.text,
    ##        )
    ##    return self

    def remove_punctuation(self):
        """Remove punctuation"""
        self.text = re.sub(r"[^\w\s]", "", self.text)
        return self

    def remove_non_ascii(self):
        """Remove non-ascii character"""
        self.text = (
            unicodedata.normalize("NFKD", self.text)
            .encode("ascii", "ignore")
            .decode("utf-8", "ignore")
        )
        return self

    def normalize_word(self):
        """Normalize slang world"""
        normal_word_path = pd.read_csv("key_norm.csv")

        self.text = " ".join(
            [
                normal_word_path[normal_word_path["singkat"] == word]["hasil"].values[0]
                if (normal_word_path["singkat"] == word).any()
                else word
                for word in self.text.split()
            ]
        )
        return self

    def stemming(self):
        """Stemming for Bahasa with Sastrawi"""
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        self.text = stemmer.stem(self.text)
        return self

    def tokenize(self):
        """Tokenize words"""
        self.words = nltk.word_tokenize(self.text)
        return self

    def stopwords_removal(self):
        """Stopword removal"""
        stopword = stopwords.words("indonesian")
        more_stopword = [
            "assalamualaikum", "wr", "wb", "pak",
            "bu", "selamat", "siang", "pagi",
            "sore", "malam", "saya",
            "terimakasih", "terima",
            "kasih", "kepada", "bpk",
            "ibu", "mohon", "tolong",
            "maaf", "dear", "wassalamualaikum",
            "regards", "nbsp", "amp", "lg", "lgi", "kak",
            "bapakibu","bapak", "admin", ]  # add more stopword to default corpus
        stop_factory = stopword + more_stopword
        stop_factory.remove('tak')
        stop_factory.remove('akhir')
        
        clean_words = []
        for word in self.words:
            if word not in stop_factory:
                clean_words.append(word)
        self.words = clean_words  
        return self

    def join_words(self):
        """Join all words"""
        self.words = " ".join(self.words)
        return self
    
    def do_all(self, text):
        """Do all text preprocessing process""" 
        self.text = text
        self = self.lowercase()
        self = self.strip_html()
        self = self.remove_url()
        self = self.remove_email()
        self = self.remove_between_square_brackets()
        self = self.remove_numbers()
        self = self.remove_emoticon()
        self = self.remove_emoji()
        ##self = self.convert_emoticon()
        ##self = self.convert_emoji()
        self = self.remove_punctuation()
        self = self.remove_non_ascii()
        self = self.normalize_word()
        self = self.stemming()
        self = self.tokenize()
        self = self.stopwords_removal()
        self = self.join_words()
        return self.words

def dask_this(data):
    data['clean_keluhan'] = data['keluhan'].apply(tp.do_all)
    return data

In [ ]:
list_sheet = ['Akademik', 'PSAL', 'PuTI', 'Logistik', 'Keuangan', 'Pengelolaan Mahasiswa', 'Pengembangan Karir, Alumni dan ', 'Laboran', 'Kemahasiswaan', 'LAA']

data_keluhan = []
data_bagian = []
for s in list_sheet:
  
  data_append = pd.read_excel(open('daftar bagian.xlsx', 'rb'),
                sheet_name=str(s))
  data_keluhan += data_append['KELUHAN'].tolist()
  data_bagian+= data_append['BAGIAN'].tolist()
  # data.append([data_append['KELUHAN'].tolist(), data_append['BAGIAN'].tolist()])
  

data = pd.DataFrame({"keluhan": data_keluhan, "bagian": data_bagian})

data

,keluhan,bagian
0,"Dear BAA,Mohon di bantu untuk merubah satatus ...",BAA
1,Persyaratan untuk pembuatan ktm terlalu rumit,BAA
2,Biaya pembuatan ktm yang terlalu mahal,BAA
3,warna dan tulisan pada ktm mudah pudar,BAA
4,Tolong saya sudah 2 kali ke bagian kemahasiswa...,BAA
...,...,...
1600,saat pemiihan kantor saat magang ada berapa ge...,LAA
1601,min sayaa sudah mengambil kantor magang di bal...,LAA
1602,magang nantinya bisa di tempatkan di kantor ya...,LAA
1603,min saya mau pindah kantor KP saya tidak betah...,LAA


In [ ]:
tp = TextPreprocessing() # load module text preprocessing  
ddata = dd.from_pandas(data, npartitions=10)
start_time = time.time()

try:
  start_time = time.time()
  data = ddata.map_partitions(dask_this).compute(scheduler='processes', num_workers=10)
except:
    print('Text preprocessing failed !')
else:
    data.to_csv('clean_data_training.csv', encoding='utf-8')
    print('Text preprocessing success !')
    print('Elapsed time:', time.time() - start_time, 'seconds')
finally:
    print('\nFinish')

dibaawha ini asal semuaa

In [ ]:
tp = TextPreprocessing() # load module text preprocessing  
ddata = dd.from_pandas(data, npartitions=10)
start_time = time.time()

try:
  start_time = time.time()
  data = ddata.map_partitions(dask_this).compute(scheduler='processes', num_workers=10)
except:
    print('Text preprocessing failed !')
else:
    data.to_csv('clean_data_training.csv', encoding='utf-8')
    print('Text preprocessing success !')
    print('Elapsed time:', time.time() - start_time, 'seconds')
finally:
    print('\nFinish')

In [ ]:
try:
  start_time = time.time()
  data = ddata.map_partitions(dask_this).compute(scheeduler='processes', num_workers=10)